<a href="https://colab.research.google.com/github/nannthd/ProjectNNGB/blob/main/%5BObject_Detection01%5D_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#การเตรียมข้อมูลภาพ (Preprocessing)
การเตรียมข้อมูลภาพ (Image Preprocessing) เป็นกระบวนการที่จำเป็นต้องทำก่อนนำภาพมาฝึกโมเดล Deep Learning หรือ Machine Learning เพื่อให้โมเดลสามารถเรียนรู้และทำนายผลได้อย่างถูกต้อง โดยขั้นตอนการเตรียมข้อมูลภาพมีหลายอย่าง ได้แก่ การปรับขนาดภาพ (Resizing), การแปลงสี (Color Conversion), การปรับความสว่างและความเข้มของภาพ (Brightness and Contrast), การเติมสี (Color Filling), การตัดภาพ (Image Cropping) เป็นต้น

In [ ]:
# อ่านไฟล์ภาพและไฟล์ติดตาม
image = cv2.imread('image.jpg')
annotation = json.load(open('annotation.json'))

# กำหนดค่าความสูงและความกว้างของภาพเพื่อใช้ในการปรับขนาด
image_height, image_width, _ = image.shape

# ปรับขนาดภาพให้เหมาะสมกับการทำงานของโมเดล
resized_image, resize_ratio = resize_image(image, max_side_length=800)

# ปรับความสว่าง ความคมชัด และคอนทราสต์ในภาพ
enhanced_image = enhance_image(resized_image)

# สร้างชุดข้อมูลภาพที่ติดตาม (annotated dataset) โดยเพิ่มขอบเขตหรือส่วนของโครโมโซมที่ต้องการนับลงในภาพ
annotated_image, annotation_info = create_annotated_image(enhanced_image, annotation)

In [ ]:
import cv2
import numpy as np

# อ่านภาพจากไฟล์
img = cv2.imread('01.jpg')

# ปรับขนาดภาพ
#img = cv2.resize(img, (224, 224))

# แปลงสีจาก BGR เป็น RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ปรับความสว่างและความเข้มของภาพ
img = cv2.convertScaleAbs(img, alpha=1.2, beta=20)

# ตัดภาพ
x = 100
y = 100
h = 300
w = 300
img = img[y:y+h, x:x+w]

# แสดงภาพ
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from google.colab.patches import cv2_imshow

img = cv2.imread('01.jpg')
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
import cv2
img = cv2.imread('01.jpg')
if img is None:
    print("Image file not found or corrupted")

Image file not found or corrupted
